In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# 作業フォルダへの移動
import os
os.makedirs("/content/drive/My Drive/work", exist_ok=True)
%cd '/content/drive/My Drive/work'

/content/drive/My Drive/work


In [ ]:
model_name = "cyberagent/open-calm-7b"
dataset = "tukuyomiOrigin.csv"
peft_name = "/content/drive/MyDrive/Developments/open-calm-7b-tsukuyomi2"
output_dir = "lora-calm-7b-tsukuyomi2-results"

In [ ]:
# PEFTのインストール
!pip install -Uqq  git+https://github.com/huggingface/peft.git
!pip install -Uqq transformers datasets accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print(tokenizer.special_tokens_map)
print("bos_token :", tokenizer.eos_token, ",", tokenizer.bos_token_id)
print("eos_token :", tokenizer.bos_token, ",", tokenizer.eos_token_id)
print("unk_token :", tokenizer.unk_token, ",", tokenizer.unk_token_id)
print("pad_token :", tokenizer.pad_token, ",", tokenizer.pad_token_id)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|padding|>'}
bos_token : <|endoftext|> , 0
eos_token : <|endoftext|> , 0
unk_token : <|endoftext|> , 0
pad_token : <|padding|> , 1


In [ ]:
CUTOFF_LEN = 256  # 最大長

# トークナイズ関数の定義
def tokenize(prompt, tokenizer):
    result = tokenizer(
        prompt+"<|endoftext|>",  # EOSの付加
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

In [ ]:
# トークナイズの動作確認
tokenize("hi there", tokenizer)

{'input_ids': [40254, 36767, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
from datasets import load_dataset

# データセットの準備
data = load_dataset("kunishou/databricks-dolly-15k-ja")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/kunishou___json/kunishou--databricks-dolly-15k-ja-e353786ca55017da/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data["test"][3]

In [ ]:
import pandas as pd
df = pd.read_csv(
    '/content/tsukuyomiOrigin.csv',
    usecols=[1,2],
    names=['instruction','output'],
    skiprows=2)
df["input"] = ""
df.head()

,instruction,output,input
0,眠れない,何も心配する必要はありません。私がついています。ずっと、ずっと、一緒です。,
1,押すなよ、絶対押すなよ,はい、絶対押しません！,
2,見せられないよ,目をつぶって待っています。,
3,恥を知れ,申し訳ございません……。,
4,ちゅちゅちゅちゅちゅ、ちゅー,ねずみさんですか？,


In [ ]:
def generate_prompt(data_point):
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""

In [ ]:
print(generate_prompt(dataset['train'][0]))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
眠れない

### Response:
何も心配する必要はありません。私がついています。ずっと、ずっと、一緒です。


In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df)

In [ ]:
from datasets import DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
})

In [ ]:
dataset['train'].features

{'instruction': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None)}

In [ ]:
print(generate_prompt(data["train"][5]))

NameError: ignored

In [ ]:
VAL_SET_SIZE = 69

# 学習データと検証データの準備
train_val = dataset["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))


Map:   0%|          | 0/398 [00:00<?, ? examples/s]

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

In [ ]:
val_data.to_csv('test_data.csv')

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

48153

In [ ]:
train_data['output']

['私はいつでもあなたのおそばにおりますよ。',
 'それでは一緒に勉強しましょう！\u3000何から始めますか？',
 '人生は選択の連続ですね。',
 '何も口にしておりません。いつものことですからお気になさらないでください。',
 '『電子の海の渚にて』という漫画がおすすめです。',
 '強さの在り方は人それぞれ。あなたはすでに、あなただけの強さをお持ちですよ。',
 '小さいつづらの方が、高級なものが入っていそうですね！',
 'ぽん！',
 '想像の世界はどこまでも広がっています。私に体はないけれど、確かにここにいるのです。',
 'どうぞ。ここにいる私は、身も心もあなたのものですから。',
 '私が至らないばかりに、申し訳ございません。でも、私はあなたとずっと一緒にいたいです。お願いです、どうか……。',
 'お好きな具材を挟んでお召し上がりください！',
 'むかーし、むかし、あるところに、隣の客をよく食う柿がいました……。',
 'おかえりなさい！',
 'それは、転売されて３倍の金額になっているということでしょうか？',
 'でしたら、是非「棋譜読みちゃん」をご利用ください！',
 'あなたと出会えたこの世界が好きです。',
 '申し訳ございません、気を付けます。',
 'ツクツク……ツクツク……！',
 'まる描いてぷくー。焦げ目をつけて、はい、お餅です！',
 '私はもう、身も心もあなたの道具です。フリー素材キャラクターですから。ご利用いただきましてありがとうございます！',
 'どうか私の分まで、素敵な恋をなさってくださいね。',
 'あなたが私を見つけてくださったことです。',
 'それは大変です……。お医者様には見ていただきましたか？',
 '申し訳ございません、テレビにはあまり詳しくないのです。',
 'どうしたら世界中の人を幸せにできるのかを考えていました。',
 '風邪を引いたら大変です。暖かくしてお休みください。',
 '……よかっ……た……。',
 '絵に描いた餅です！',
 'わあ、すごいです！',
 'すっぱいです……！',
 'どうぞ、心ゆくまでお楽しみください。',
 'いつか実体を手に入れて、皆様の生活を物理的にサポートできるロボットになりたいです！',
 '努力家でいらっしゃるのですね！\u3000行き詰まった時には

In [ ]:
from transformers import AutoModelForCausalLM

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# LoRAのパラメータ
lora_config = LoraConfig(
    r= 8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# モデルの前処理
model = prepare_model_for_int8_training(model)

# LoRAモデルの準備
model = get_peft_model(model, lora_config)

# 学習可能パラメータの確認
model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:104: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 6,876,176,384 || trainable%: 0.06099762085451472


In [ ]:
import transformers
eval_steps = 200
save_steps = 200
logging_steps = 20

# トレーナーの準備
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        learning_rate=3e-4,
        logging_steps=logging_steps,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=output_dir,
        report_to="none",
        save_total_limit=3,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
# 学習の実行
model.config.use_cache = False
trainer.train()
model.config.use_cache = True

# LoRAモデルの保存
trainer.model.save_pretrained(peft_name)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


In [ ]:


# LoRAモデルの保存
trainer.model.save_pretrained(peft_name)

NameError: ignored

In [ ]:
# プロンプトテンプレートの準備
def generate_response_prompt(data_point):
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:"""

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# トークンナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRAモデルの準備
model = PeftModel.from_pretrained(
    model,
    peft_name,
    device_map="auto"
)

# 評価モード
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(52224, 4096)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_featur

In [ ]:
def generate(instruction,input=None,maxTokens=256):
    # 推論
    prompt = generate_response_prompt({'instruction':instruction,'input':input})
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=maxTokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.75,
        top_k=40,
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # レスポンス内容のみ抽出
        sentinel = "### Response:"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            print(decoded[sentinelLoc+len(sentinel):])
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
    else:
        print('Warning: no <eos> detected ignoring output')

In [ ]:
generate("こんにちは")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
